<a href="https://colab.research.google.com/github/ashok2216-A/bharat-intern_internship-tasks/blob/main/Task1_Stock_Price_Prediction_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinancials

**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
# plt.style.use('dark_background')
plt.rc('figure', figsize=(18,8))
plt.rc('lines', markersize=1)
plt.rc('font', size=10)
import warnings
warnings.filterwarnings('ignore')
import datetime
from datetime import datetime
import yfinance as yf
from yahoofinancials import YahooFinancials
!rm -rf ./logs/
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
%load_ext tensorboard
from keras.models import Sequential # For creating a sequential neural network model
from keras.layers import Dense, Dropout, LSTM, Bidirectional

**Import Data**

In [ ]:
data = yf.download('GOOG', start='2015-01-01', end=datetime.now(), progress=True)
data.head(10)

**Dataset Information**

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
print('Duplicate: ',data.duplicated().sum())

In [ ]:
data.info()

In [ ]:
data.describe().astype('float')

**Ploting Relationship between Opening and Closing Stock**

In [ ]:
plt.title('Google Stock Price ($)')
plt.xlabel('Open Stock Price ($)')
plt.ylabel('Close Stock Price ($)')
plt.scatter(data['Open'],data['Close'])
plt.show()

**Visualizing Opening and Closing Stock Price**

In [ ]:
data[['Open', 'Close']].plot()
plt.xticks(rotation=45)
plt.xlabel('Date',size=15)
plt.ylabel('Stock Price',size=15)
plt.title('Google Stock Closing Price and Opening Price($)',size=15)
plt.show()

**Visualizing Overall Dataframe**

In [ ]:
data[['Open', 'Close', 'High', 'Low', 'Adj Close']].plot()
plt.xticks(rotation=45)
plt.xlabel('Date',size=15)
plt.ylabel('Stock Price',size=15)
plt.title('Google Stock Closing Price and Opening Price($)',size=15)
plt.show()

In [ ]:
data = data[['Open', 'Close']]
data.head(3)

**Doing Feature Engineering**

In [ ]:
MMS = MinMaxScaler()
data[data.columns] = MMS.fit_transform(data)
data

**Spliting the Data**

In [ ]:
training_size = round(len(data) * 0.75)
print(training_size)
train_data = data[:training_size]
test_data = data[training_size:]
print(train_data.shape, test_data.shape)

**Sequencing the Data**

In [ ]:
def create_sequence(dataset):
  sequences = []
  labels = []

  start_idx = 0

  for stop_idx in range(50,len(dataset)): # Selecting 50 rows at a time
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return (np.array(sequences),np.array(labels))

In [ ]:
train_seq, train_label = create_sequence(train_data)
test_seq, test_label = create_sequence(test_data)
train_seq.shape, train_label.shape, test_seq.shape, test_label.shape

**Model Selection**

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_seq.shape[1], train_seq.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

In [ ]:

model.fit(train_seq, train_label, epochs=15,validation_data=(test_seq, test_label), verbose=1)

In [ ]:
test_predicted = model.predict(test_seq)
test_predicted[:5]

In [ ]:
test_inverse_predicted = MMS.inverse_transform(test_predicted)
test_inverse_predicted[:5]

In [ ]:
df_merge = pd.concat([data.iloc[-test_seq.shape[0]:].copy(),
                          pd.DataFrame(test_inverse_predicted,columns=['open_predicted','close_predicted'],
                                       index=data.iloc[-test_seq.shape[0]:].index)], axis=1)

In [ ]:
df_merge[['Open','Close']] = MMS.inverse_transform(df_merge[['Open','Close']])
df_merge.head()

In [ ]:
# plotting the actual open and predicted open prices on date index
df_merge[['Open','open_predicted']].plot(figsize=(10,6))
plt.xticks(rotation=45)
plt.xlabel('Date',size=15)
plt.ylabel('Stock Price',size=15)
plt.title('Actual vs Predicted for open price',size=15)
plt.show()

In [ ]:
df_merge[['Close','close_predicted']].plot(figsize=(10,6))
plt.xticks(rotation=45)
plt.xlabel('Date',size=15)
plt.ylabel('Stock Price',size=15)
plt.title('Actual vs Predicted for close price',size=15)
plt.show()

**Adding Extra 10 Days for future Prediction**

In [ ]:
df_merge = df_merge.append(pd.DataFrame(columns=df_merge.columns,
                                        index=pd.date_range(start=df_merge.index[-1], periods=11, freq='D', closed='right')))
df_merge['2023-09-15':'2023-10-25']

In [ ]:
upcoming_prediction = pd.DataFrame(columns=['open','close'],index=df_merge.index)
upcoming_prediction.index=pd.to_datetime(upcoming_prediction.index)

In [ ]:
curr_seq = test_seq[-1:]

for i in range(-10,0):
  up_pred = model.predict(curr_seq)
  upcoming_prediction.iloc[i] = up_pred
  curr_seq = np.append(curr_seq[0][1:],up_pred,axis=0)
  curr_seq = curr_seq.reshape(test_seq[-1:].shape)

In [ ]:
upcoming_prediction[['open','close']] = MMS.inverse_transform(upcoming_prediction[['open','close']])

**Visualizing Predicted Data**

In [ ]:
# plotting Upcoming Open price on date index
fig,ax=plt.subplots(figsize=(10,5))
ax.plot(df_merge.loc['2023-06-25':,'Open'],label='Current Open Price')
ax.plot(upcoming_prediction.loc['2023-9-15':,'open'],label='Upcoming Open Price')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
ax.set_xlabel('Date',size=15)
ax.set_ylabel('Stock Price',size=15)
ax.set_title('Upcoming Open price prediction',size=15)
ax.legend()
fig.show()

In [ ]:
# plotting Upcoming Open price on date index
fig,ax=plt.subplots(figsize=(10,5))
ax.plot(df_merge.loc['2023-06-25':,'Close'],label='Current Close Price')
ax.plot(upcoming_prediction.loc['2023-9-15':,'close'],label='Upcoming Close Price')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
ax.set_xlabel('Date',size=15)
ax.set_ylabel('Stock Price',size=15)
ax.set_title('Upcoming Close price prediction',size=15)
ax.legend()
fig.show()